In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import os
from os import listdir
from os.path import isfile
import pickle

import tensorflow as tf
import numpy as np
import sklearn
from sklearn import svm

FLAGS = None
SAVE_PATH = '../../logdir/models'

model = 'mnist_cff_2000.ckpt'
SAVE_PATH = '../logdir/models'
ELITE_LOC = '../logdir/elites/mnist_test_elites_19:45:10_18-09-17'
DIAG_DIR = os.path.join(ELITE_LOC, 'diagrams')
TRUE_DIR = os.path.join(DIAG_DIR, 'true')
FALSE_DIR = os.path.join(DIAG_DIR, 'elites')

TAKE = 100

In [2]:
true_files = [f for f in listdir(TRUE_DIR) if isfile(os.path.join(TRUE_DIR, f))]
false_files = [f for f in listdir(FALSE_DIR) if isfile(os.path.join(FALSE_DIR, f))]

dummy = np.genfromtxt(os.path.join(TRUE_DIR, true_files[0]), delimiter=',').reshape(1,-1)
trues = np.empty(shape=(len(true_files), TAKE*2))
for i in range(len(true_files)):
    temp = np.genfromtxt(os.path.join(TRUE_DIR, true_files[i]), delimiter=',')
    trues[i, :] = np.append(temp[:TAKE,0], temp[:TAKE,1], axis=0)
falses = np.empty(shape=(len(false_files),TAKE*2))
for i in range(len(false_files)):
    temp = np.genfromtxt(os.path.join(FALSE_DIR, false_files[i]), delimiter=',')
    falses[i, :] = np.append(temp[:TAKE,0], temp[:TAKE,1], axis=0)

totes = np.vstack((trues, falses)).reshape((-1,TAKE*2), order='F')
totes_labels = np.empty(shape=(totes.shape[0], 1))
totes_labels[0::2, :] = 1
totes_labels[1::2, :] = 0

train_im = totes[:-100,:]
train_lab = totes_labels[:-100,:]

test_im = totes[-100:,:]
test_lab = totes_labels[-100:,:]

In [3]:
print(train_im.shape, train_lab.shape)
clf = svm.SVC()
clf.fit(train_im, train_lab)

(3040, 200) (3040, 1)


/home/tgebhart/.virtualenvs/tf_activation/local/lib/python2.7/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [4]:
preds = clf.predict(test_im)

In [5]:
sklearn.metrics.accuracy_score(test_lab,preds)

0.52000000000000002

In [6]:
reg = sklearn.linear_model.LogisticRegression()
reg.fit(train_im, train_lab)
predreg = reg.predict(test_im)

In [7]:
sklearn.metrics.accuracy_score(test_lab,predreg)

0.46000000000000002